# ГЕНЕРАЦИЯ ТЕКСТОВ С ПОМОЩЬЮ ЦЕПЕЙ МАРКОВА

пример генерации гороскопов

Данильченко Вадим

In [1]:
import re
import random
from collections import deque

In [82]:
# загрузим тексты гороскопов
text = open('../data/гороскопы/гороскопы.txt', encoding='utf-8').read().lower()
text = re.sub(r'\.', ' .', text)
text = re.sub(r'\:', ' :', text)
text = text.replace("»", " END").replace("«", "").replace("\n", " ").replace(".", " END").replace(",", "")
frags = [f.lower() if f != 'END' else f for f in text.split(" ") if f]
print(len(frags))

11623


In [83]:
# словарь, который хранит окна в качестве ключа в паре «(ключ, значение)» и распределения в качестве значений в этой паре.
# проводится подсчет элементов, чтобы для обращения к какому-либо из них не нужно было пробегать каждый раз по всему набору данных.
# присутствуют функции для возврата случайного слова. Одна функция выбирает случайный ключ в словаре, а другая, 
# принимая во внимание число появлений каждого слова в тексте, возвращает нужное нам слово.
class Dictogram(dict):
    def __init__(self, iterable=None):
        # Инициализируем наше распределение как новый объект класса, 
        # добавляем имеющиеся элементы
        super(Dictogram, self).__init__()
        self.types = 0  # число уникальных ключей в распределении
        self.tokens = 0  # общее количество всех слов в распределении
        if iterable:
            self.update(iterable)

    def update(self, iterable):
        # Обновляем распределение элементами из имеющегося 
        # итерируемого набора данных
        for item in iterable:
            if item in self:
                self[item] += 1
                self.tokens += 1
            else:
                self[item] = 1
                self.types += 1
                self.tokens += 1

    def count(self, item):
        # Возвращаем значение счетчика элемента, или 0
        if item in self:
            return self[item]
        return 0

    def return_random_word(self):
        random_key = random.sample(self, 1)
        # Другой способ:
        # random.choice(histogram.keys())
        return random_key[0]

    def return_weighted_random_word(self):
        # Сгенерировать псевдослучайное число между 0 и (n-1),
        # где n - общее число слов
        random_int = random.randint(0, self.tokens-1)
        index = 0
#         list_of_keys = 
        # вывести 'случайный индекс:', random_int
        for key in self.keys():
            index += self[key]
            # вывести индекс
            if(index > random_int):
                # вывести list_of_keys[i]
                return key

In [84]:
# если данные не существуют, создаем дикторграмму, иначе просто добавляем к текущему
def make_markov_model(data):
    markov_model = dict()
    for i in range(0, len(data)-1):
        if data[i] in markov_model:
            # Просто присоединяем к уже существующему распределению
            markov_model[data[i]].update([data[i+1]])
        else:
            markov_model[data[i]] = Dictogram([data[i+1]])
    return markov_model

In [85]:
# генерация контента, основанная на текущем состоянии - находим все ключи «END» и выбираем слово, следующее за одним из них
# после генерации начального слова ищем, какое слово может идти дальше, обращаясь к тому же словарю, 
# и выбираем нужное на основании комбинации вероятности и случайности. 
# повторяем пока предложение не достигнет установленной нами длины, и в конце возвращаем его. 
def generate_random_start(model):
    # для генерации любого начального слова:
    # return random.choice(model.keys())

    # будем использовать те слова, что являлись началом предложений в корпусе
    if 'END' in model:
        seed_word = 'END'
        while seed_word == 'END':
            seed_word = model['END'].return_weighted_random_word()
        return seed_word
    return random.choice(list(model.keys()))


def generate_random_sentence(length, markov_model, current_word=None):
    if not current_word:
        current_word = generate_random_start(markov_model)
    sentence = [current_word]
    for i in range(0, length):
        current_dictogram = markov_model[current_word]
        random_weighted_word = current_dictogram.return_weighted_random_word()
        current_word = random_weighted_word
        sentence.append(current_word)
    sentence[0] = sentence[0].capitalize()
    return ' '.join(sentence) + '.'
#     return sentence

In [86]:
# инициализируем модель
model = make_markov_model(frags)

In [87]:
generate_random_sentence(50, model, 'рыбам').replace(" END", ".")

'Рыбам состоящим в эпоху водолея – не только в этот период. проблемы и становится много работать по полной программе. актуальна всегда вызывали зависть у самых перспективных. неожиданно возобновятся старые долги расплачиваться по декабрь 2020 год постоянных перемен. однако забегая вперед не ранее декабря или стран и стабильность.'

In [92]:
generate_random_sentence(50, model, 'раков').replace(" END", ".")

'Раков ждет удача будет не стоит опасаться не проиграйте! любовь семья - в конце лета и если есть вся первая половина года. сохраняйте равновесие и уже во второй половине года возникнут напряженные моменты во многих семьях реальны перемены в своих неудачах и нужды не ожидается во второй половины мая..'

In [106]:
generate_random_sentence(50, model, 'лев').replace(" END", ".")

'Лев : наступает время – это окажется вся первая половина года – о разводе. в этом направлении. 2020 года гораздо менее интересен возможно и совсем неплохо сложится январь – ее инициаторов понятна – враги и до декабря или в делах или попросит примирения бывший супруг и задуматься об окончательном.'

In [114]:
generate_random_sentence(50, model, 'львов').replace(" END", ".")

'Львов ждет удача в год находится в другом варианте возникнут проблемы которые могут приобрести дом дачу семейный бизнес - в этом случае крупные расходы возрастут а если супруги уже отработавшего себя. гораздо более беспокойна. нестандартный подход и устойчивой власти у супругов во второй половине года окажется квартира дом а.'

In [116]:
generate_random_sentence(50, model, 'тельцам').replace(" END", ".")

'Тельцам улыбнется удача на все неприятности в роли миротворцев на коллег. чтобы после наслаждаться заслуженным успехом предыдущего времени чтобы попробовать все это цикл соединений сатурна и на рф будет продвижение на вторую половину 2020 год менее значимой нежели работа поможет избежать ошибок. козам можно сказать стрессовой половине года многие.'

In [130]:
generate_random_sentence(50, model, 'овен').replace(" END", ".")

'Овен. возможна удачная поездка в период. и долгожданным подарком. для влюбленных трудно что грядут серьезные эмоциональные переживания и наслаждаться жизнью. наступит период когда покажется что делать и мечталось то что многое в ином более скромном варианте агрессивно и на самотек. в 2021 год принесет больших денег.'

не увлекаюсь чтением гороскопов, но думаю там примерно так и пишут..)